In [21]:
import sys, os, pickle
sys.path.append('/scratch/oem214/vanilla-rtrl/')
from cluster import write_job_file, submit_job, unpack_processed_data
from continual_learning import *
from core import *
from dynamics import *
from functions import *
from gen_data import *
from learning_algorithms import *
from optimizers import *
from plotting import *
from wrappers import *

In [58]:
### --- SET UP ALL CONFIGS --- ###
from itertools import product
n_seeds = 40
macro_configs = config_generator(lr=[0.01, 0.001],
                                 L2_reg=[0.001, 0.0001])
micro_configs = tuple(product(macro_configs, list(range(n_seeds))))

### --- SELECT PARTICULAR CONFIG --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [59]:
task = Flip_Flop_Task(3, 0.05, input_magnitudes=None)
N_train = 50000
N_test = 5000
checkpoint_interval = 100
sigma = 0

data = task.gen_data(N_train, N_test)

n_in = task.n_in
n_hidden = 32
n_out = task.n_out
W_in  = np.random.normal(0, np.sqrt(1/(n_in)), (n_hidden, n_in))
W_rec = np.random.normal(0, np.sqrt(1/n_hidden), (n_hidden, n_hidden))
W_out = np.random.normal(0, np.sqrt(1/(n_hidden)), (n_out, n_hidden))

b_rec = np.zeros(n_hidden)
b_out = np.zeros(n_out)

alpha = 1
sigma = 0

rnn = RNN(W_in, W_rec, W_out, b_rec, b_out,
          activation=tanh,
          alpha=alpha,
          output=identity,
          loss=mean_squared_error)

learn_alg = RFLO(rnn, alpha=alpha, L2_reg=params['L2_reg'], L1_reg=0.0001)
optimizer = SGD_Momentum(lr=params['lr'], mu=0.6)

monitors = ['rnn.loss_']

sim = Simulation(rnn)
sim.run(data, learn_alg=learn_alg, optimizer=optimizer,
        sigma=sigma,
        monitors=monitors,
        verbose=True,
        report_accuracy=False,
        report_loss=True,
        checkpoint_interval=checkpoint_interval)

Progress: 10.0% complete 
Time Elapsed: 1.3s 
Average loss: 0.13208912243691104 
Test loss: 0.28236694545560964 

Progress: 20.0% complete 
Time Elapsed: 2.7s 
Average loss: 0.015647852047229387 
Test loss: 0.00944128687831115 

Progress: 30.0% complete 
Time Elapsed: 4.2s 
Average loss: 0.00208442787941905 
Test loss: 0.03563428913399282 

Progress: 40.0% complete 
Time Elapsed: 5.7s 
Average loss: 0.0015816778161949015 
Test loss: 0.004647485058953659 

Progress: 50.0% complete 
Time Elapsed: 7.1s 
Average loss: 0.000977543969697607 
Test loss: 0.003918337437198527 

Progress: 60.0% complete 
Time Elapsed: 8.6s 
Average loss: 0.0008298068420096915 
Test loss: 0.003983271857754357 

Progress: 70.0% complete 
Time Elapsed: 10.0s 
Average loss: 0.0008817492113148464 
Test loss: 0.004349873946086811 

Progress: 80.0% complete 
Time Elapsed: 11.5s 
Average loss: 0.0007762040536410945 
Test loss: 0.0040429245997072386 

Progress: 90.0% complete 
Time Elapsed: 12.9s 
Average loss: 0.0007472

In [ ]:
test_sim = Simulation(rnn)
test_sim.run(data, mode='test', monitors=['rnn.loss_'], verbose=False)
processed_data = sim.mons['rnn.loss_']

In [ ]:
### --- SAVE RESULTS -- ###
result = {'sim': sim, 'i_seed': i_seed, 'task': task,
          'config': params, 'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_'+str(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [60]:
###Truncate file above
file_name = 'pipeline_test'
job_name = 'scratch'
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py ../cluster_main_scripts/{}.py'.format(file_name, job_name))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook pipeline_test.ipynb to script
[NbConvertApp] Writing 3859 bytes to pipeline_test.py
awk: 1: unexpected character '.'


In [61]:
###Submit job to cluster
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(job_name))
submit_job('../job_scripts/scratch.s', n_jobs)

sending incremental file list

sent 10,357 bytes  received 45 bytes  20,804.00 bytes/sec
total size is 1,214,259  speedup is 116.73
Submitted batch job 7553038


In [63]:
###Get job status
get_ipython().system('squeue -u oem214')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7552619        cm  jupyter   oem214  R    1:06:47      1 cm01


In [64]:
configs_array, results_array, key_order, sim_dict = unpack_processed_data('../job_scripts/scratch.s')